In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split #dividir el de prueba y entrenamiento como Candy
from sklearn.preprocessing import StandardScaler #para poder escalar todas las variables numéricas (independientes), ya que vienen en dimensiones o escalas diferentes

In [86]:
df = pd.read_csv('Francia_limpio.csv')
df.head()

,Unnamed: 0,name,host_name,host_since,host_location,host_about,host_response_time,host_is_superhost,host_neighbourhood,host_identity_verified,...,number_of_reviews_l30d,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_location,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_entire_homes.1,calculated_host_listings_count_shared_rooms,reviews_per_month
0,0,"Duplex avec vue, centre ville",Isabelle,2010-10-24,"Lyon, France",I'm 47 years old. I enjoy books and arts. I'm ...,within a few hours,f,5th Arrondissement,t,...,0.0,4.70,4.71,4.78,4.86,1.0,1.0,1.0,0.0,0.33
1,1,Nice Flat in the center of Lyon,Clementine,2011-02-26,"Lyon, France",Je travaille dans le cinéma et je voyage beauc...,within an hour,f,5th Arrondissement,t,...,1.0,4.75,4.80,4.96,4.88,1.0,1.0,1.0,0.0,0.15
2,2,"Centre of old town,home sweet home",Polali,2011-06-03,"Lyon, France","J'ai 34 ans , je suis animatrice auprès de per...",a few days or more,f,5th Arrondissement,t,...,0.0,4.48,4.27,4.72,4.86,1.0,1.0,1.0,0.0,0.82
3,3,"Bel apartment, 2 rooms, Lyon center",Sandrine,2011-06-06,"Lyon, France","We are a family with 3 daughters, aged 16-14-1...",within an hour,f,1st Arrondissement,t,...,0.0,4.78,4.89,4.89,4.95,1.0,1.0,1.0,0.0,0.12
4,4,Quartier Terreaux 4/6 pers. Hyper centre / Clim,Giuseppina,2011-07-18,"Rhône-Alpes, France","Mon pays d'origine est l'Italie, mais je résid...",within an hour,f,1st Arrondissement,t,...,2.0,4.62,4.86,4.83,4.88,2.0,1.0,1.0,0.0,2.99


In [87]:
df1 = df.copy()

## 1. host_identity_verified

In [3]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['availability_365','host_acceptance_rate','host_total_listings_count']] #numérica
Var_Dep = df['host_identity_verified'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [4]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], shape=(2968,), dtype=object)

In [ ]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='t') 
print('Precisión del modelo "t":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='f') 
print('Precisión del modelo "f":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='t')
print('Sensibilidad del modelo "t":')
print(sensibilidad)

#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='f')
print('Sensibilidad del modelo "f":')
print(sensibilidad)

Matriz de Confusión
[[   0  383]
 [   0 2585]]
Precisión del modelo "t":
0.8709568733153639
Precisión del modelo "f":
0.0
Exactitud del modelo:
0.8709568733153639
Sensibilidad del modelo "t":
1.0
Sensibilidad del modelo "f":
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 2. host_response_time

**Convertirla a dicotómica**

In [6]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
unico

array(['a few days or more', 'within a day', 'within a few hours',
       'within an hour'], dtype=object)

In [7]:
#Convertimos una variable categórica a dicotómica 
df ['host_response_time']= df['host_response_time'].replace(['within a day', 'within a few hours','within an hour'],'a day')

In [8]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['host_response_rate','host_acceptance_rate','host_listings_count']] #numérica
Var_Dep = df['host_response_time'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [9]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['a day', 'a day', 'a day', ..., 'a day', 'a day', 'a day'],
      shape=(2968,), dtype=object)

In [ ]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='a day') 
print('Precisión del modelo "a day":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='a few days or more') 
print('Precisión del modelo "a few days or more":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='a day')
print('Sensibilidad del modelo "a day":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='a few days or more')
print('Sensibilidad del modelo "a few days or more":')
print(sensibilidad)

Matriz de Confusión
[[2833    0]
 [ 135    0]]
Precisión del modelo "a day":
0.9545148247978437
Precisión del modelo "a few days or more":
0.0
Exactitud del modelo:
0.9545148247978437
Sensibilidad del modelo "a day":
1.0
Sensibilidad del modelo "a few days or more":
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 3. host_is_superhost

**Convertir a dicotómica**

In [11]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_is_superhost'])
unico

array(['Desconocido', 'f', 't'], dtype=object)

In [12]:
#Convertimos una variable categórica a dicotómica 
df ['host_is_superhost']= df['host_is_superhost'].replace(['Desconocido'],'f')

In [13]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['availability_365','number_of_reviews','host_listings_count']] #numérica
Var_Dep = df['host_is_superhost'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [14]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 't', 'f'], shape=(2968,), dtype=object)

In [ ]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='f') 
print('Precisión del modelo "f":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='t') 
print('Precisión del modelo "t":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='f')
print('Sensibilidad del modelo "f":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='t')
print('Sensibilidad del modelo "t":')
print(sensibilidad)

Matriz de Confusión
[[2316   79]
 [ 526   47]]
Precisión del modelo "f":
0.8149190710767066
Precisión del modelo "t":
0.373015873015873
Exactitud del modelo:
0.7961590296495957
Sensibilidad del modelo "f":
0.9670146137787057
Sensibilidad del modelo "t":
0.08202443280977312


## 4. room_type

**Convertir a dicotómica**

In [16]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
unico

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [17]:
#Convertimos una variable categórica a dicotómica 
df ['room_type']= df['room_type'].replace(['Hotel room','Private room','Shared room'],'Public or shared room')

In [18]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['review_scores_rating','accommodates','price']] #numérica
Var_Dep = df['room_type'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [19]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'Entire home/apt', 'Entire home/apt'],
      shape=(2968,), dtype=object)

In [ ]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Entire home/apt') 
print('Precisión del modelo "Entire home/apt":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Public or shared room') 
print('Precisión del modelo "Public or shared room":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Entire home/apt')
print('Sensibilidad del modelo "Entire home/apt":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Public or shared room')
print('Sensibilidad del modelo "Public or shared room":')
print(sensibilidad)

Matriz de Confusión
[[2278   21]
 [ 534  135]]
Precisión del modelo "Entire home/apt":
0.810099573257468
Precisión del modelo "Public or shared room":
0.8653846153846154
Exactitud del modelo:
0.8130053908355795
Sensibilidad del modelo "Entire home/apt":
0.9908655937364071
Sensibilidad del modelo "Public or shared room":
0.20179372197309417


## 5. host_listings_count

**Convertir a dicotómica**

In [21]:
#Limite superior e inferior de columna objetivo age
Max = df['host_listings_count'].max()
Min = df['host_listings_count'].min()
Limites =[Min,Max]
Limites

[np.float64(1.0), np.float64(3.0)]

In [ ]:
#Intervalos
intervalos = np.linspace(0.9,3.1,3) #se agrega un decimal mas/menos para que agarre el rango
intervalos

#Creamos las categorías
categorias = ['Una propiedad','Mas de una']

#Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['host_listings_count']= pd.cut(x=df['host_listings_count'],bins= intervalos,labels=categorias)
df['host_listings_count']

In [23]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['maximum_nights','accommodates','price']] #numérica
Var_Dep = df['host_listings_count'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [24]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Una propiedad', 'Una propiedad', 'Una propiedad', ...,
       'Una propiedad', 'Una propiedad', 'Una propiedad'],
      shape=(2968,), dtype=object)

In [ ]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Una propiedad') 
print('Precisión del modelo "Una propiedad":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Mas de una') 
print('Precisión del modelo "Mas de una":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Una propiedad')
print('Sensibilidad del modelo "Una propiedad":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Mas de una')
print('Sensibilidad del modelo "Mas de una":')
print(sensibilidad)

Matriz de Confusión
[[   0  142]
 [   0 2826]]
Precisión del modelo "Una propiedad":
0.9521563342318059
Precisión del modelo "Mas de una":
0.0
Exactitud del modelo:
0.9521563342318059
Sensibilidad del modelo "Una propiedad":
1.0
Sensibilidad del modelo "Mas de una":
0.0


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## 6. bedrooms

**Convertir a dicotómica**

In [26]:
#Limite superior e inferior de columna objetivo age
Max = df['bedrooms'].max()
Min = df['bedrooms'].min()
Limites =[Min,Max]
Limites

[np.float64(0.0), np.float64(3.0)]

In [ ]:
#Intervalos
intervalos = np.linspace(-1,3.1,3) #se agrega un decimal mas/menos para que agarre el rango
intervalos

#Creamos las categorías
categorias = ['Una habitacion','Mas de una']

#Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['bedrooms']= pd.cut(x=df['bedrooms'],bins= intervalos,labels=categorias)
df['bedrooms']

In [28]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['maximum_nights','accommodates','price']] #numérica
Var_Dep = df['bedrooms'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [29]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Una habitacion', 'Una habitacion', 'Una habitacion', ...,
       'Una habitacion', 'Mas de una', 'Una habitacion'],
      shape=(2968,), dtype=object)

In [31]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Una habitacion') 
print('Precisión del modelo "Una habitación":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Mas de una') 
print('Precisión del modelo "Más de una":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Una habitacion')
print('Sensibilidad del modelo "Una habitación":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Mas de una')
print('Sensibilidad del modelo "Más de una":')
print(sensibilidad)

Matriz de Confusión
[[ 339  487]
 [  67 2075]]
Precisión del modelo "Una habitación":
0.8099141295862607
Precisión del modelo "Más de una":
0.8349753694581281
Exactitud del modelo:
0.8133423180592992
Sensibilidad del modelo "Una habitación":
0.9687208216619981
Sensibilidad del modelo "Más de una":
0.41041162227602906


## 7. beds

**Convertir a dicotómica**

In [32]:
#Limite superior e inferior de columna objetivo age
Max = df['beds'].max()
Min = df['beds'].min()
Limites =[Min,Max]
Limites

[np.float64(0.0), np.float64(3.0)]

In [ ]:
#Intervalos
intervalos = np.linspace(-1,3.1,3) #se agrega un decimal mas/menos para que agarre el rango
intervalos

#Creamos las categorías
categorias = ['Una cama','Mas de una']

#Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['beds']= pd.cut(x=df['beds'],bins= intervalos,labels=categorias)
df['beds']

In [35]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['maximum_nights','accommodates','price']] #numérica
Var_Dep = df['beds'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [36]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Una cama', 'Una cama', 'Una cama', ..., 'Mas de una', 'Una cama',
       'Una cama'], shape=(2968,), dtype=object)

In [37]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Una cama') 
print('Precisión del modelo "Una cama":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Mas de una') 
print('Precisión del modelo "Más de una":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Una cama')
print('Sensibilidad del modelo "Una cama":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Mas de una')
print('Sensibilidad del modelo "Más de una":')
print(sensibilidad)

Matriz de Confusión
[[ 254  532]
 [ 204 1978]]
Precisión del modelo "Una cama":
0.7880478087649403
Precisión del modelo "Más de una":
0.5545851528384279
Exactitud del modelo:
0.7520215633423181
Sensibilidad del modelo "Una cama":
0.9065077910174152
Sensibilidad del modelo "Más de una":
0.3231552162849873


## 8. number_of_reviews_l30d

**Convertir a dicotómica**

In [50]:
#Limite superior e inferior de columna objetivo age
Max = df['number_of_reviews_l30d'].max()
Min = df['number_of_reviews_l30d'].min()
Limites =[Min,Max]
Limites

[np.float64(0.0), np.float64(2.0)]

In [ ]:
#Intervalos
intervalos = np.linspace(-1,2.1,3) #se agrega un decimal mas/menos para que agarre el rango
intervalos

#Creamos las categorías
categorias = ['No tiene','Ultimos 30 dias']

#Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['number_of_reviews_l30d']= pd.cut(x=df['number_of_reviews_l30d'],bins= intervalos,labels=categorias)
df['number_of_reviews_l30d']

In [53]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['availability_365','host_acceptance_rate','price']] #numérica
Var_Dep = df['number_of_reviews_l30d'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [54]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['No tiene', 'Ultimos 30 dias', 'No tiene', ..., 'No tiene',
       'No tiene', 'No tiene'], shape=(2968,), dtype=object)

In [55]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='No tiene') 
print('Precisión del modelo "No tiene":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Ultimos 30 dias') 
print('Precisión del modelo "Ultimos 30 dias":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='No tiene')
print('Sensibilidad del modelo "No tiene":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Ultimos 30 dias')
print('Sensibilidad del modelo "Ultimos 30 dias":')
print(sensibilidad)

Matriz de Confusión
[[2382   34]
 [ 539   13]]
Precisión del modelo "No tiene":
0.8154741526874358
Precisión del modelo "Ultimos 30 dias":
0.2765957446808511
Exactitud del modelo:
0.8069407008086253
Sensibilidad del modelo "No tiene":
0.9859271523178808
Sensibilidad del modelo "Ultimos 30 dias":
0.02355072463768116


## 9.availability_30

**Convertir a dicotómica**

In [61]:
#Limite superior e inferior de columna objetivo age
Max = df['availability_30'].max()
Min = df['availability_30'].min()
Limites =[Min,Max]
Limites

[np.int64(0), np.int64(30)]

In [ ]:
#Intervalos
intervalos = np.linspace(-1,30.1,3) #se agrega un decimal mas/menos para que agarre el rango
intervalos

#Creamos las categorías
categorias = ['Hasta 14 dias','De 14 a 30 dias']

#Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df['availability_30']= pd.cut(x=df['availability_30'],bins= intervalos,labels=categorias)
df['availability_30']

In [65]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df[['number_of_reviews','host_acceptance_rate','price']] #numérica
Var_Dep = df['availability_30'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [66]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Hasta 14 dias', 'Hasta 14 dias', 'Hasta 14 dias', ...,
       'Hasta 14 dias', 'Hasta 14 dias', 'Hasta 14 dias'],
      shape=(2968,), dtype=object)

In [67]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Hasta 14 dias') 
print('Precisión del modelo "Hasta 14 dias":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='De 14 a 30 dias') 
print('Precisión del modelo "De 14 a 30 dias":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Hasta 14 dias')
print('Sensibilidad del modelo "Hasta 14 dias":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='De 14 a 30 dias')
print('Sensibilidad del modelo "De 14 a 30 dias":')
print(sensibilidad)

Matriz de Confusión
[[  33  918]
 [  30 1987]]
Precisión del modelo "Hasta 14 dias":
0.6839931153184166
Precisión del modelo "De 14 a 30 dias":
0.5238095238095238
Exactitud del modelo:
0.6805929919137467
Sensibilidad del modelo "Hasta 14 dias":
0.985126425384234
Sensibilidad del modelo "De 14 a 30 dias":
0.03470031545741325


## 10. accommodates

**Convertir a dicotómica**

In [88]:
#Limite superior e inferior de columna objetivo age
Max = df1['accommodates'].max()
Min = df1['accommodates'].min()
Limites =[Min,Max]
Limites

[np.float64(1.0), np.float64(7.0)]

In [ ]:
#Intervalos
intervalos = np.linspace(0.9,7.1,3) #se agrega un decimal mas/menos para que agarre el rango
intervalos

#Creamos las categorías
categorias = ['Hasta 4 personas','De 4 a 7 personas']

#Ajustar maximo de filas
pd.options.display.max_rows = None

#Finalmente creamos las categorías en la columna numérica
df1['accommodates']= pd.cut(x=df1['accommodates'],bins= intervalos,labels=categorias)
df1['accommodates']

In [90]:
#Declaramos las variables dependientes e independientes para regresión logística
#Dependiente es dicotómica y las independientes de tipo numérico
Vars_Indep = df1[['beds','price','bedrooms']] #numérica
Var_Dep = df1['accommodates'] #dicotómica

#Redefinir
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en entrenamiento y prueba
#test size es que queremos el 30%, podemos irle cambiando
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.3, random_state =None) #Va a asignar aleatoriamente 

In [91]:
#Se escalan los datos
escalar = StandardScaler()

X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algortimo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Emtrenamos el modelo
algoritmo.fit(X_train,y_train)
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['Hasta 4 personas', 'De 4 a 7 personas', 'De 4 a 7 personas', ...,
       'Hasta 4 personas', 'De 4 a 7 personas', 'De 4 a 7 personas'],
      shape=(2968,), dtype=object)

In [92]:
#Verifico la matriz de confunsión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix (y_test, y_pred)
print('Matriz de Confusión')
print(matriz)

#Calculo la precisión Var 1
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='Hasta 4 personas') 
print('Precisión del modelo "Hasta 4 personas":')
print(precision)

#Calculo la precisión Var 2
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_pred,average='binary',pos_label='De 4 a 7 personas') 
print('Precisión del modelo "De 4 a 7 personas":')
print(precision)

#Calculo la exactitud
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test,y_pred)
print('Exactitud del modelo:')
print(exactitud)

#Calculo la sensibilidad del modelo Var 1
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='Hasta 4 personas')
print('Sensibilidad del modelo "Hasta 4 personas":')
print(sensibilidad)

#Calculo la sensibilidad del modelo Var 2
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test,y_pred, average ='binary',pos_label='De 4 a 7 personas')
print('Sensibilidad del modelo "De 4 a 7 personas":')
print(sensibilidad)

Matriz de Confusión
[[ 818  346]
 [ 220 1584]]
Precisión del modelo "Hasta 4 personas":
0.8207253886010363
Precisión del modelo "De 4 a 7 personas":
0.7880539499036608
Exactitud del modelo:
0.8092991913746631
Sensibilidad del modelo "Hasta 4 personas":
0.8780487804878049
Sensibilidad del modelo "De 4 a 7 personas":
0.7027491408934707
